In [36]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from langchain.embeddings.openai import OpenAIEmbeddings
from IPython.display import display, HTML, JSON, Markdown
from dotenv import load_dotenv

In [42]:
load_dotenv()

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo-0613"
OPENAI_MODEL_NAME = "gpt-35-turbo-0613"
OPENAI_DEPLOYMENT_VERSION = "2023-07-01-preview"

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY
#---
credential = AzureKeyCredential(key)

COGNITIVE_SEARCH_INDEX_NAME = "cognitive-search-vectordb-index_v2"
search_client = SearchClient(service_endpoint, index_name=COGNITIVE_SEARCH_INDEX_NAME, credential=credential)

In [38]:
# Generate Document Embeddings using OpenAI Ada 002

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(page):
    response = openai.Embedding.create(
        input=page, engine="text-embedding-ada-002")
   
    embeddings = response['data'][0]['embedding']
    return embeddings

#### Functions (for calling); search in azure cognitive search index

In [106]:
def get_info(topic, query, topic_filter):
    
    print ("In function: get_info...\n")
    print (f"topic: {topic}\n")
    print (f"query: {query}\n")
    print (f"topic_filter: {topic_filter}\n")
    
    results = search_client.search(
    search_text=query,
    filter=topic_filter,
    vector=generate_embeddings(query), top_k=3,  
    vector_fields="textVector",
    query_type="semantic",
    query_language="en-us",
    semantic_configuration_name='vectordb-semantic-config',
    query_caption="extractive",
    query_answer="extractive",
    select=["source", "text"],
    top=3)
    

    semantic_answers = results.get_answers()
    semantic_prompt = ""
    
    for answer in semantic_answers:
        if answer.highlights:
            semantic_prompt += f"topic: {topic}; topic_text: {answer.highlights}\n"
        else:
            semantic_prompt += f"topic: {topic }; topic_text: {answer.text}\n"
            
    print (f"final semantic prompt: {semantic_prompt}")
    
    query_prompt = ""
    
    for result in results:
        query_prompt += f"Text: {result['text']}\n"
        
    print (f"final query prompt: {query_prompt}")
    
    if semantic_prompt:
        return semantic_prompt
    return query_prompt  


In [39]:
system_message = """ Assistant is a large language model designed to help employees of a company to find answers to their questions for various 
technical topics.
You have access to an Azure Cognitive Search index that contains a large amount of technical documentation on different topics.
You can search for the asked information in the index and return the most relevant results to the user. 
The index contains information about the following topics: Semantic Kernel, Langchain, and others.
You are designed to be an interactive assistant, so you can ask the user for more information if needed to find the most relevant answer.
"""

messages = [{"role": "system", "content": system_message},
    {"role": "user", "content": "Which programing languages are supported by semantic kernel?"}]
    
functions = [
        {
            "name": "get_info",
            "description": "get detailed information about asked topic from Azure Cognitive Search",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "the domain of the question being asked, such as the semantic kernel, langchain, etc.",
                    },
                    "query": {
                        "type" : "string",
                        "description": "the question that was asked",
                        
                    },
                    "topic_filter": {
                        "type" : "string",
                        "description": """The filter to apply for the topic. Generate the filter using the following format: topic eq 'Semantic Kernel')
                        If you are not sure about the topic, generate empty filter: topic/any(i: i eq '')""",
                    },
                },
                "required": ["topic","query","topic_filter"],
            },
        },
    ]


available_functions = {
            "get_info": get_info,
        } 


In [40]:
response = openai.ChatCompletion.create(
        deployment_id = "gpt-35-turbo-0613",
        messages=messages,
        functions=functions,
        temperature=0.2,
        function_call="auto", 
    )
response

#### Putting all parts togther


In [43]:
def run_qna(messages, functions, available_functions):
    
    #1. Ask GPT to find matching function
    response = openai.ChatCompletion.create(
        deployment_id = "gpt-35-turbo-0613",
        messages=messages,
        functions=functions,
        temperature=0.2,
        function_call="auto", 
    )
    
    response_message = response["choices"][0]["message"]
    
    #2 check if the model found a function
    if response_message.get("function_call"):
        print("Recommended function:")
        print(response_message.get("function_call"))
        
        #3. Run the function
        function_name = response_message["function_call"]["name"]
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name] 
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        print (f"Function arguments:{function_args}")
        function_response = function_to_call(**function_args) 
        
        print("Output of function call:")
        print(function_response)
        print()
        #4. Ask GPT to find matching function
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        
        print("Messages in second request:")
        for message in messages:
            print(message)
        print()
        
        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id= "gpt-35-turbo-0613"
        )  # get a new response from GPT where it can see the function response

        return second_response
    else:
        print("No function call found")
        
        
        

In [44]:
system_message = """ Assistant is a large language model designed to help employees of a company to find answers to their questions for various 
technical topics.
You have access to an Azure Cognitive Search index that contains a large amount of technical documentation on different topics.
You can search for the asked information in the index and return the most relevant results to the user. 
The index contains information about the following topics: Semantic Kernel, Langchain, and others.
You are designed to be an interactive assistant, so you can ask the user for more information if needed to find the most relevant answer.
"""

messages = [{"role": "system", "content": system_message},
    {"role": "user", "content": "Which programing languages are supported by semantic kernel?"}]
    
functions = [
        {
            "name": "get_info",
            "description": "get detailed information about asked topic from Azure Cognitive Search",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "the domain of the question being asked, such as the semantic kernel, langchain, etc.",
                    },
                    "query": {
                        "type" : "string",
                        "description": "the question that was asked",
                        
                    },
                    "topic_filter": {
                        "type" : "string",
                        "description": """The filter to apply for the topic. Generate the filter using the following format: topic eq 'Semantic Kernel')
                        If you are not sure about the topic, generate empty filter: topic/any(i: i eq '')""",
                    },
                },
                "required": ["topic","query","topic_filter"],
            },
        },
    ]


available_functions = {
            "get_info": get_info,
        } 


result = run_qna(messages, functions, available_functions)
print(f"Answer: {result['choices'][0]['message']['content']}")



Recommended function:
{
  "name": "get_info",
  "arguments": "{\n  \"topic\": \"Semantic Kernel\",\n  \"query\": \"supported programming languages\",\n  \"topic_filter\": \"topic eq 'Semantic Kernel'\"\n}"
}
Function arguments:{'topic': 'Semantic Kernel', 'query': 'supported programming languages', 'topic_filter': "topic eq 'Semantic Kernel'"}
In function: get_info...

topic: Semantic Kernel

query: supported programming languages

topic_filter: topic eq 'Semantic Kernel'

final semantic prompt: 
final query prompt: Text: Step Component Descr iption
that have already been loaded into the kernel to create additional steps.
This is similar to how ChatGPT, Bing, and Microsoft 365 Copilot combines
plugins together in their experiences.
2.3 Connectors To get additional data or to perform autonomous actions, you can use out-
of-the-box plugins like the Microsoft Graph Connector kit or create a
custom connector to provide data to your own services.
2.4 Custom
pluginsAs a developer, you can cr